# Example 6: Using VIX rate-of-change as a signal to trade E-Mini S&P 500 futures

## Pre-requisites
### 1. If you have not opened the notebook in Colab, select the button below
<a href="https://githubtocolab.com/SIGTechnologies/sigtech-python/blob/master/examples/6_Trading_ESPMini500_Futures_Using_VIX_Signals.ipynb">
    <img src="https://sigtech.com/wp-content/uploads/2023/08/grey_google_colab.svg"></a>

### 2. Get a FRED API key
This notebook uses the FREDAPI library and requires you to have an API key for Federal Reserve Economic Data. Get one [here](https://fred.stlouisfed.org/docs/api/api_key.html).

### 3. Enter your Sigtech and FRED API keys
After pasting in your API keys, you need to run the cell. In Colab, hover your cursor over an individual code cell and click play to run it.
>**Tip**!\
>After pasting in your API keys, you can press `CTRL-F9` (Windows) or `⌘-F9` (Mac) to run the entire notebook at once.

In [ ]:
# Install FRED API and our Python SDK
%pip install sigtech 
%pip install fredapi

# Import OS and our Python SDK
import sigtech.api as sig
import os

# Define your API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# <YOUR_API_KEY> with the API key you have generated. e.g. os.environ['SIGTECH_API_KEY'] = 'sig_A1B2C3D4E5f6g7h8i9'
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

# Define your FRED API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# FRED_API_KEY with the API key you have generated. e.g. fred = Fred(api_key="0011223344556677")
from fredapi import Fred
fred = Fred(api_key=os.environ["FRED_API_KEY"])

### 4. Set up your Colab environment

In [ ]:
# Install our Python SDK and empyrical
%pip install empyrical

# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import numpy as np
import empyrical as ep
import matplotlib.pyplot as plt

# Set any parameters 
plt.rcParams['figure.figsize'] = [16, 8]

### 5. Create a session
After installing our Python SDK, defining your API key, importing any additional Python libraries or functions you require, and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to VIX ROC signal strategies

This strategy is a short-term trading approach that aims to capitalize on market volatility by using the VIX (CBOE Volatility Index) and its rate-of-change (ROC) as key indicators. The strategy involves generating trading signals based on the VIX's 10-day ROC and subsequently executing trades in E-Mini S&P 500 futures contracts to take advantage of market movements.

## Our strategy
- We will fetch historical VIX data using the Federal Reserve Economic Data (FRED) API. 
- We will calculate the 10-day ROC of the VIX (i.e. the percentage change in the VIX over the past 10 days) and based on this ROC, we will set predefined threshold values for generating trading signals.
- Using these signals we will create a signals strategy and backtest the performance of trading E-mini S&P 500 futures using VIX ROC as a trading signal. 
- We will then compare the performance of our strategy against a benchmark rolling futures strategy for E-mini S&P 500 futures by plotting the performance of both strategies and using the `empyrical` library to compare various financial metrics for both strategies. 

## 1. Calculate the rate-of-change (ROC)

First, we will fetch historial data from FRED and use this to calculate the 10 day ROC. 

In [ ]:
# Fetch data from FRED
vix = fred.get_series('VIXCLS')

# Calculate the 10-day ROC
roc = (vix - vix.shift(10)) / vix.shift(10)

## 2. Generate trading signals

First, we set upper thresholds for when to hold E-mini S&P 500 futures and when to short them. When the ROC crosses above the `upper_threshold_flat` or below the `upper_threshold_short`, it indicates a favorable market condition, and the strategy generates a "flat" signal, holding the existing position. If the ROC exceeds the `upper_threshold_short` it indicates higher market volatility, and the strategy generates a "short" signal, taking a bearish position.


In [ ]:
# Define thresholds for generating trading signals
upper_threshold_flat = 0.2   # example value
upper_threshold_short = 0.75   # example value

# Generate trading signals based on ROC
signals = pd.Series(1, index=roc.index)
signals[(roc > upper_threshold_flat) & (roc <= upper_threshold_short)] = 0  # flat signal
signals[roc > upper_threshold_short] = -1  # short signal

# Visualise the change in ROC over time
roc.plot()

## 3. Backtest the VIX ROC signals strategy and compare it to a benchmark

Now that we have the required components, we can create both a benchmark rolling futures strategy and our VIX ROC signals strategy. First, let's create the benchmark strategy:

In [ ]:
es = sig.RollingFutureStrategy(
    contract_code='ES',
    contract_sector='INDEX',
    start_date='2010-01-10'
)

Then, for our VIX ROC signals strategy, we need to create a DataFrame that can be used in our SDK's `SignalStrategy` class.

> A `SignalStrategy` requires a `signal_input`. This is a pandas DataFrame where the column headers are the instrument names and the values are the signals for each of the instruments. These signals can be either a number of units *or* a weight.

In [ ]:
signal_df = pd.DataFrame({es.name:signals})
signal_df.tail()

Finally, we can create our E-mini S&P 500 signals strategy.

In [ ]:
s = sig.SignalStrategy(
    currency='USD',
    signal_input=signal_df,
    start_date='2010-01-10',
    rebalance_frequency='1BD',
)

## 4. Compare the performance of the two strategies

Firstly, we plot the performance of the two strategies.

In [ ]:
es.history().plot(label="E-mini S&P 500 benchmark rolling futures strategy")
s.history().plot(label="E-mini S&P 500 VIX ROX signals strategy")
plt.legend()
plt.show()

Then, we use the `empyrical` library to view some of the standard performance metrics for a financial strategy.

In [ ]:
def calculate_metrics(returns, factor_returns, strategy_name):
    # Calculate metrics
    alpha = ep.alpha(returns, factor_returns)
    beta = ep.beta(returns, factor_returns)
    sharpe_ratio = ep.sharpe_ratio(returns)
    sortino_ratio = ep.sortino_ratio(returns)
    annual_volatility = ep.annual_volatility(returns)
    annual_return = ep.annual_return(returns)
    max_drawdown = ep.max_drawdown(returns)
    omega_ratio = ep.omega_ratio(returns)
    tail_ratio = ep.tail_ratio(returns)
    cagr = ep.cagr(returns)

    # Create a DataFrame
    metrics = pd.DataFrame({
        'Alpha': [alpha],
        'Beta': [beta],
        'Sharpe Ratio': [sharpe_ratio],
        'Sortino Ratio': [sortino_ratio],
        'Annual Volatility': [annual_volatility],
        'Annual Return': [annual_return],
        'Max Drawdown': [max_drawdown]
    }, index=[strategy_name]).transpose()

    return metrics


In [ ]:
# Assuming 's.history()' and 'es.history()' are your strategy's and ES's return series respectively
strategy_returns = s.history().pct_change().dropna()
es_returns = es.history().pct_change().dropna()

# Calculate the metrics
strategy_metrics = calculate_metrics(strategy_returns, es_returns, 'VIX Signal Strategy')
es_metrics = calculate_metrics(es_returns, es_returns, 'E-Mini S&P 500')

# Concatenate the metrics
all_metrics = pd.concat([strategy_metrics, es_metrics], axis=1)

all_metrics
